
# 线性回归的简洁实现

随着深度学习框架的发展，开发深度学习应用变得越来越便利。实践中，我们通常可以用比上一节更简洁的代码来实现同样的模型。在本节中，我们将介绍如何使用tensorflow2.0推荐的keras接口更方便地实现线性回归的训练。

## 生成数据集

我们生成与上一节中相同的数据集。其中`features`是训练数据特征，`labels`是标签。

In [1]:
import tensorflow as tf

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = tf.random.normal(shape=(num_examples, num_inputs), stddev=1)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += tf.random.normal(labels.shape, stddev=0.01)

虽然tensorflow2.0对于线性回归可以直接拟合，不用再划分数据集，但我们仍学习一下读取数据的方法

In [2]:
from tensorflow import data as tfdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = tfdata.Dataset.from_tensor_slices((features, labels))
# 随机读取小批量
dataset = dataset.shuffle(buffer_size=num_examples)
dataset = dataset.batch(batch_size)
data_iter = iter(dataset)

In [3]:
for X, y in data_iter:
    print(X, y)
    break

tf.Tensor(
[[ 0.11906993  1.853482  ]
 [ 1.4858664  -0.18852489]
 [-1.0006745  -0.40935215]
 [-0.25300497  0.60063976]
 [-0.28377545 -1.5488006 ]
 [ 0.27551156 -0.61813927]
 [-0.80954224  0.77936345]
 [-0.06401849 -0.0905276 ]
 [ 0.17151324  1.3873678 ]
 [ 1.2723187   0.66405845]], shape=(10, 2), dtype=float32) tf.Tensor(
[-1.8673204   7.8212767   3.5861506   1.657495    8.903213    6.8579035
 -0.06758562  4.3810105  -0.16158912  4.485695  ], shape=(10,), dtype=float32)


定义模型,tensorflow 2.0推荐使用keras定义网络，故使用keras定义网络
我们先定义一个模型变量`model`，它是一个`Sequential`实例。
在keras中，`Sequential`实例可以看作是一个串联各个层的容器。
在构造模型时，我们在该容器中依次添加层。
当给定输入数据时，容器中的每一层将依次计算并将输出作为下一层的输入。
重要的一点是，在keras中我们无须指定每一层输入的形状。
因为为线性回归，输入层与输出层全连接，故定义一层

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import initializers as init
model = keras.Sequential()
model.add(layers.Dense(1, kernel_initializer=init.RandomNormal(stddev=0.01)))

定义损失函数和优化器：损失函数为mse，优化器选择sgd随机梯度下降
在keras中，定义完模型后，调用`compile()`方法可以配置模型的损失函数和优化方法。定义损失函数只需传入`loss`的参数，keras定义了各种损失函数，并直接使用它提供的平方损失`mse`作为模型的损失函数。同样，我们也无须实现小批量随机梯度下降，只需传入`optimizer`的参数，keras定义了各种优化算法，我们这里直接指定学习率为0.01的小批量随机梯度下降`tf.keras.optimizers.SGD(0.03)`为优化算法

In [5]:
from tensorflow import losses
loss = losses.MeanSquaredError()

In [6]:
from tensorflow.keras import optimizers
trainer = optimizers.SGD(learning_rate=0.03)

In [7]:
loss_history = []

在使用keras训练模型时，我们通过调用`model`实例的`fit`函数来迭代模型。`fit`函数只需传入你的输入x和输出y，还有epoch遍历数据的次数，每次更新梯度的大小batch_size, 这里定义epoch=3，batch_size=10。
使用keras甚至完全不需要去划分数据集

In [8]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for (batch, (X, y)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            l = loss(model(X, training=True), y)
        
        loss_history.append(l.numpy().mean())
        grads = tape.gradient(l, model.trainable_variables)
        trainer.apply_gradients(zip(grads, model.trainable_variables))
    
    l = loss(model(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.numpy().mean()))
    

epoch 1, loss: 0.577524
epoch 2, loss: 0.010238
epoch 3, loss: 0.000279


下面我们分别比较学到的模型参数和真实的模型参数。我们可以通过model的`get_weights()`来获得其权重（`weight`）和偏差（`bias`）。学到的参数和真实的参数很接近。

In [9]:
true_w, model.get_weights()[0]

([2, -3.4], array([[ 1.9945697],
        [-3.3903365]], dtype=float32))

In [10]:
true_b, model.get_weights()[1]

(4.2, array([4.1920047], dtype=float32))

In [11]:
loss_history

[39.801727,
 53.19048,
 14.46788,
 39.947865,
 24.486965,
 30.713726,
 39.17563,
 13.047705,
 33.28475,
 20.761524,
 18.420874,
 34.008083,
 14.405246,
 17.041443,
 7.005189,
 18.382719,
 18.695303,
 13.632373,
 12.635169,
 20.074924,
 15.94778,
 9.27402,
 10.427583,
 8.54941,
 16.81535,
 6.8781943,
 15.035431,
 11.794742,
 11.860071,
 3.8811011,
 7.9945374,
 7.8730416,
 9.143669,
 10.242535,
 5.7133584,
 12.0080595,
 5.0345087,
 5.8301377,
 6.3047495,
 7.8470206,
 10.113134,
 9.405206,
 8.132036,
 5.154071,
 4.421684,
 5.987176,
 5.059777,
 5.080542,
 3.4634635,
 2.47151,
 5.6223817,
 4.015649,
 3.8196254,
 3.3439813,
 3.7710037,
 3.2606869,
 0.9403268,
 1.5239546,
 2.5819716,
 3.9055488,
 6.8448067,
 2.0272841,
 3.6449275,
 4.1734414,
 3.8974013,
 1.9378322,
 3.0132825,
 2.4585395,
 1.1937939,
 1.9572703,
 1.2297701,
 0.69520515,
 1.5201247,
 1.3764166,
 1.4739829,
 1.7456497,
 0.68866235,
 2.0207152,
 1.0318539,
 1.1638341,
 1.3316251,
 0.6911205,
 1.9045906,
 1.5170772,
 0.81408226